# Preliminaries and Dataframe Construction

In [1]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import random

#Import Encounters from Database Query
df = pd.read_pickle("encounters.pkl")

#Formatting
plt.rcParams['font.family'] = 'Times New Roman'  # Set plt shows font to Times New Roman
plt.rcParams['axes.grid'] = True  # Ensure line graphs display on graphs
sns.set_palette(sns.color_palette('Set2')) #set color palette to a nice seaborn style https://seaborn.pydata.org/tutorial/color_palettes.html

### Monte Carlo Model for 50% Capacity
Extending the approach of Bhavani et al., our model simulates an n/20 (i.e. 5%, 10%...95% capacity, where n = <20>) ventilator shortage by: (1) randomly sampling 20 patients from our dataset, and assigning them to ten pairs (A and B), (2) if n>10, assigning ventilators to *both* patients in the first n/2 pairs (i.e. A=1, B=1), and if n≤10 assigning no ventilator to both patients in n/2 pairs (i.e. A=0, B=0), (3) ranking the individuals in each remaining pair for priority based on the relevant protocol (i.e. A>B), 4) assigning a ventilator to the highest priority patient in each pair (e.g. A=1, B=0), and 4) repeating this process until all patients in our dataset have been assigned to receive or not receive a ventilator.

The input is a datafrae that includes EncounterID, Age (as a tiebreaker for Bhavani), Protocol Score (SOFA Score, NY SOFA Band, Colorado Score, Bhavani Score, Age_Group), and Actual_Survival outcome.

The output is a dataframe that includes Capacity, Protocol, Run, EncounterID, Allocated, Survived.

For these simulations, we complete 1000 runs per protocol.

In [6]:
### Testing Simulation ###

df_small_TEST = (df
      .rename(columns={'Survived':'Actual_Survival'})
      .reindex(columns=['EncounterID', 'Age', 'Maryland_Score', 'Actual_Survival'])
)
#MC for different degrees of scarcity with NY Protocol
beds = 1
patients = 2 #patients fixed level across simulations
decisions = 3700 #number of monte carlo samples (i.e. ) per MC simulation
runs = 10 #number of MC simulations per capacity level

results_TEST_50 = pd.DataFrame()

for i in range(runs): #iterate over each run
      sample = (df_small_TEST
      .sample(n=decisions, replace=False) #randomly shuffle dataframe
      .assign(Protocol = 'Maryland', Run = i+1, Capacity = (beds/patients), Allocated=0, Survived = 0)
      .assign(Decision = lambda df_1:
                  np.arange(len(df_1)) // patients + 1
            )
      .assign(Rank=lambda df_2: df_2
                  .sort_values('Age')
                  .groupby(['Decision'])['Maryland_Score'].rank(method="first")
            )
      .assign(Allocated=lambda df_3: 
                  df_3['Allocated'].mask(df_3['Rank'] <= beds, 1),
            Survived = lambda df_3: 
                  df_3['Survived'].mask(df_3['Rank'] <= beds, df_3['Actual_Survival'])
            )
      )
      results_TEST_50 = pd.concat([results_TEST_50, sample])

#results_TEST_50.to_csv('MC_TEST_50.csv', index=False)

### PRODUCTION Monte Carlo Sim (50% Scarcity)
Following Code Blocks generate 1000 runs of simulation for 
- (1) Pure SOFA, 
- (2) NY SOFA, 
- (3) Age-Groups, 
- (4) Lottery, 
- (5) Bhavani Multi-Principle,
- (6) Colorado (modified)

The NY SOFA protocol assigns priority tiers by SOFA score (SOFA ≤7 = Tier 1; SOFA 8-11 = Tier 2; SOFA ≥12 = Tier 3), with ties broken through a lottery.

In [3]:
#NY Protocol (50% Scarcity)
df_small_ny = (df
      .rename(columns={'Survived':'Actual_Survival'})
      .reindex(columns=['EncounterID', 'NY_Score', 'Actual_Survival'])
)
#MC for different degrees of scarcity with NY Protocol
beds = 1
patients = 2 #patients fixed level across simulations
decisions = 3700 #number of monte carlo samples (i.e. ) per MC simulation
runs = 1000 #number of MC simulations per capacity level

results_NY_50 = pd.DataFrame()

for i in range(runs): #iterate over each run
      sample = (df_small_ny
      .sample(n=decisions, replace=False) #randomly shuffle dataframe
      .assign(Protocol = 2, Run = i+1, Capacity = (beds/patients), Allocated=0, Survived = 0)
      .assign(Decision = lambda df_1:
                  np.arange(len(df_1)) // patients + 1
            )
      .assign(Rank=lambda df_2: 
                  df_2.groupby(['Decision'])['NY_Score'].rank(method="first")
            )
      .assign(Allocated=lambda df_3: 
                  df_3['Allocated'].mask(df_3['Rank'] <= beds, 1),
            Survived = lambda df_3: 
                  df_3['Survived'].mask(df_3['Rank'] <= beds, df_3['Actual_Survival'])
            )
      )
      results_NY_50 = pd.concat([results_NY_50, sample])

results_NY_50.to_csv('MC_NY_50.csv', index=False)


**Monte Carlo (Age-Banding)**

The age-banding protocol assigns priority by 10-year age bins, with ties broken by lottery. Age bands are '<25', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84', '>85'

In [4]:
#Age, 50% Scarcity
df_small_age = (df
      .rename(columns={'Survived':'Actual_Survival'})
      .reindex(columns=['EncounterID', 'Age_Group', 'Actual_Survival'])
)
#MC for different degrees of scarcity with NY Protocol
beds = 1
patients = 2 #patients fixed level across simulations
decisions = 3700 #number of monte carlo samples (i.e. ) per MC simulation
runs = 1000 #number of MC simulations per capacity level

results_Age_50 = pd.DataFrame()

for i in range(runs): #iterate over each run
      sample = (df_small_age
      .sample(n=decisions, replace=False) #randomly shuffle dataframe
      .assign(Protocol = 3,
            Run = i+1, 
            Capacity = (beds/patients), 
            Allocated=0, 
            Survived = 0, 
            )
      .assign(Decision = lambda df_1:
                  np.arange(len(df_1)) // patients + 1, 
            Age_Group_N = lambda df_1:
                  df_1['Age_Group'].map({'<25': 1, '25-34': 2, '35-44': 3, '45-54': 4, '55-64': 5, '65-74': 6, '75-84': 7, '>85': 8}).astype(int)
            )
      .assign(Rank=lambda df_2: 
                  df_2.groupby(['Decision'])['Age_Group_N'].rank(method="first")
            )
      .assign(Allocated=lambda df_3: 
                  df_3['Allocated'].mask(df_3['Rank'] <= beds, 1),
            Survived = lambda df_3: 
                  df_3['Survived'].mask(df_3['Rank'] <= beds, df_3['Actual_Survival'])
            )
      .drop(['Age_Group_N'], axis=1)
      )
      results_Age_50 = pd.concat([results_Age_50, sample])

results_Age_50.to_csv('MC_Age_50.csv', index=False)

**Lottery Protocol**

The lottery protocol assigns a random number between 0 and 5000 to every patient, and then assigns priority to the patient in each pair who possesses the higher number.

In [5]:
#Lottery, 50% Scarcity
df_small_lott = (df
      .rename(columns={'Survived':'Actual_Survival'})
      .reindex(columns=['EncounterID', 'Actual_Survival'])
)
#MC for different degrees of scarcity with NY Protocol
beds = 1
patients = 2 #patients fixed level across simulations
decisions = 3700 #number of monte carlo samples (i.e. ) per MC simulation
runs = 1000 #number of MC simulations per capacity level

results_Lott_50 = pd.DataFrame()

for i in range(runs): #iterate over each run
      sample = (df_small_lott
      .sample(n=decisions, replace=False) #randomly shuffle dataframe
      .assign(Protocol = 1, Run = i+1, Capacity = (beds/patients), Allocated=0, Survived = 0)
      .assign(Random = np.random.randint(0,5000,size=3700), #generate random integer
            Decision = lambda df_1:
                  np.arange(len(df_1)) // patients + 1
            )
      .assign(Rank=lambda df_2: 
                  df_2.groupby(['Decision'])['Random'].rank(method="first")
            )
      .assign(Allocated=lambda df_3: 
                  df_3['Allocated'].mask(df_3['Rank'] <= beds, 1),
            Survived = lambda df_3: 
                  df_3['Survived'].mask(df_3['Rank'] <= beds, df_3['Actual_Survival'])
            )
      )
      results_Lott_50 = pd.concat([results_Lott_50, sample])

results_Lott_50.to_csv('MC_Lott_50.csv', index=False)

**Maryland, 2017**

The Maryland, 2017 protocol assigns points by SOFA bands (<9 SOFA = 1 pt; 9-11 SOFA = 2 pts; 12-14 SOFA = 3 pts; >14 SOFA = 4pts) and adds +3 points to the SOFA score of any patient with a “severe” comorbidity (understood as a van Walraven acute Elixhauser score >=4, i.e. the upper tertile of Elixhauser scores with a “90% predicted 1 year mortality” in Snow et al.).(7,12) Ties between patients in the Maryland protocol where then broken by course-grained age-bands (0-49, 50-69, 70-84, ≥85) and then by lottery. 

In [4]:
#Maryland, 50% Scarcity
df_small_Maryland = (df
      .rename(columns={'Survived':'Actual_Survival'})
      .reindex(columns=['EncounterID', 'Maryland_Score', 'Age', 'Actual_Survival'])
      .assign(Maryland_Age = lambda df_1:
                  pd.cut(df_1['Age'], [0, 50, 70, 85, np.inf], labels=[1, 2, 3, 4])#.astype(int)
      )
)
#MC for different degrees of scarcity with NY Protocol
beds = 1
patients = 2 #patients fixed level across simulations
decisions = 3700 #number of monte carlo samples (i.e. ) per MC simulation
runs = 1000 #number of MC simulations per capacity level

results_Maryland_50 = pd.DataFrame()

for i in range(runs): #iterate over each run
      sample = (df_small_Maryland
      .sample(n=decisions, replace=False) #randomly shuffle dataframe
      .assign(Protocol = 4, Run = i+1, Capacity = (beds/patients), Allocated=0, Survived = 0)
      .assign(Decision = lambda df_1:
                  np.arange(len(df_1)) // patients + 1
            )
      .assign(Rank=lambda df_2: df_2
                  #.sort_values('Maryland_Age')
                  .groupby(['Decision'])['Maryland_Score'].rank(method="first")
            )
      .assign(Allocated=lambda df_3: 
                  df_3['Allocated'].mask(df_3['Rank'] <= beds, 1),
            Survived = lambda df_3: 
                  df_3['Survived'].mask(df_3['Rank'] <= beds, df_3['Actual_Survival'])
            )
      )
      results_Maryland_50 = pd.concat([results_Maryland_50, sample])

results_Maryland_50.to_csv('MC_Maryland_50.csv', index=False)

**Colorado**

The Colorado protocol assigns points by SOFA band (≤5 SOFA = 1pts; 6-9 SOFA = 2 pts; 10-12 SOFA = 3pts, >12 SOFA = 4 pts), and by a modified Charlson comorbidity index, with ties broken by lottery (see p100 and Appendix E, p120 of Colorado et.al.).(11) 

In the original Colorado protocol, they suggested a modified SOFA score that excluded Creatinine sub-components. Because we did not have access to SOFA sub-components, we used the full SOFA score.

In [5]:
#Colorado, 50% Scarcity
df_small_colorado = (df
      .rename(columns={'Survived':'Actual_Survival'})
      .reindex(columns=['EncounterID', 'Colorado_Score', 'Actual_Survival'])
)
#MC for different degrees of scarcity with NY Protocol
beds = 1
patients = 2 #patients fixed level across simulations
decisions = 3700 #number of monte carlo samples (i.e. ) per MC simulation
runs = 1000 #number of MC simulations per capacity level

results_Colorado_50 = pd.DataFrame()

for i in range(runs): #iterate over each run
      sample = (df_small_colorado
      .sample(n=decisions, replace=False) #randomly shuffle dataframe
      .assign(Protocol = 5, Run = i+1, Capacity = (beds/patients), Allocated=0, Survived = 0)
      .assign(Decision = lambda df_1:
                  np.arange(len(df_1)) // patients + 1
            )
      .assign(Rank=lambda df_2: 
                  df_2.groupby(['Decision'])['Colorado_Score'].rank(method="first")
            )
      .assign(Allocated=lambda df_3: 
                  df_3['Allocated'].mask(df_3['Rank'] <= beds, 1),
            Survived = lambda df_3: 
                  df_3['Survived'].mask(df_3['Rank'] <= beds, df_3['Actual_Survival'])
            )
      )
      results_Colorado_50 = pd.concat([results_Colorado_50, sample])

results_Colorado_50.to_csv('MC_Colorado_50.csv', index=False)

**Pure SOFA**

In [6]:
#Pure SOFA, 50% Scarcity
df_small_sofa = (df
      .rename(columns={'Survived':'Actual_Survival'})
      .reindex(columns=['EncounterID', 'InitialSOFA', 'Actual_Survival'])
)
#MC for different degrees of scarcity with NY Protocol
beds = 1
patients = 2 #patients fixed level across simulations
decisions = 3700 #number of monte carlo samples (i.e. ) per MC simulation
runs = 1000 #number of MC simulations per capacity level

results_sofa_50 = pd.DataFrame()

for i in range(runs): #iterate over each run
      sample = (df_small_sofa
      .sample(n=decisions, replace=False) #randomly shuffle dataframe
      .assign(Protocol = 6, Run = i+1, Capacity = (beds/patients), Allocated=0, Survived = 0)
      .assign(Decision = lambda df_1:
                  np.arange(len(df_1)) // patients + 1
            )
      .assign(Rank=lambda df_2: 
                  df_2.groupby(['Decision'])['InitialSOFA'].rank(method="first")
            )
      .assign(Allocated=lambda df_3: 
                  df_3['Allocated'].mask(df_3['Rank'] <= beds, 1),
            Survived = lambda df_3: 
                  df_3['Survived'].mask(df_3['Rank'] <= beds, df_3['Actual_Survival'])
            )
      )
      results_sofa_50 = pd.concat([results_sofa_50, sample])

results_sofa_50.to_csv('MC_sofa_50.csv', index=False)